<a href="https://www.kaggle.com/code/debajyotidas/llmtourplanner?scriptVersionId=235077733" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Building an Intelligent, AI-Powered Trip Planner, combining OpenAI and LangChain for Personalized Trip Planning

## Introduction	
In this project, we've created a sophisticated travel planning assistant that leverages OpenAI's GPT-3.5 model and LangChain's conversation management capabilities. Our system combines real-time weather data, local attractions information, and personalized preferences to generate detailed travel itineraries.

## Key Features
- Personalized itinerary generation based on:
    - Destination
    - Date range  
    - Budget constraints
    - Personal interests
    - Local weather conditions
    - Popular tourist attractions
- Interactive refinement through natural conversation
- Integration with Google Maps API for location data
- Real-time weather updates via OpenWeather API

## Technical Implementation
The system is built using:
- OpenAI's gpt-3.5-turbo-0125 model for natural language processing
- LangChain for managing conversation history and context
- Google Maps API for location and attraction data
- OpenWeather API for weather forecasts
- Python libraries: pandas, ipywidgets

## How It Works
1. Users input their travel preferences including destination, dates, and interests
2. The system fetches real-time data about local attractions and weather
3. An AI-generated itinerary is created based on all inputs
4. Users can refine the itinerary through natural conversation with the AI

## Practical Applications
This tool demonstrates the power of combining:
- Large Language Models (LLMs)
- Real-time data integration
- Conversational AI
- API orchestration

Perfect for travel agencies, individual travelers, or anyone looking to plan a detailed, personalized trip.

#### In the below cells we will look to leverage the capabilities of OpenAI's "gpt-3.5-turbo-0125" model to build a smart Trip-planner, which takes as input a trip destination of your choice, trip start and end dates, and your estimated budget for the trip. The planner also takes into consideration some of the things you maybe interested in doing on your trip, from a predefined list of interests.
#### In addition, using LangChains chaining capabilities, you can also chat with the planner to further fine-tune the generated itinenary 

In [3]:
# installing libraries
!pip install openai
!pip install langchain-openai
!pip install langchain_community
!pip install langchain
!pip install requests
!pip install googlemaps
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 9.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Suc

In [4]:
#importing dependencies
import openai
import googlemaps
import requests
import ipywidgets as widgets

from IPython.display import display, clear_output
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from openai import OpenAI

In [5]:
#Getting the API Keys for OpenAI, Google Maps, and OpenWeather
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
OPENAIKEY = user_secrets.get_secret("OPENAIKEY")
googlemaps_api_key = user_secrets.get_secret("googlemaps_api_key")
openweather_api_key = user_secrets.get_secret("openweather_api_key")

In [6]:
def get_attractions(googlemapsapi, destination):
    """Fetches a list of tourist attractions near a given destination using the Google Maps API.

    Args:
        destination (str): The name of the destination (e.g., "Paris, France").

    Returns:
        list[str]: A list of names of tourist attractions near the destination.
                   Returns an empty list if no attractions are found.

    Raises:
        ValueError: If the destination cannot be geocoded.
        googlemaps.exceptions.ApiError: If there is an issue with the Google Maps API request.
        KeyError: If the API key is not found in Kaggle secrets.

    Note:
        This function requires the 'googlemaps' libraries to be installed
        and a valid Google Maps API key stored in secrets under the key
        "googlemaps_api_key". It searches for attractions within a 5km radius
        of the geocoded destination. Requires 'googlemaps' libraries to be imported.
    """
    lat_lng = (destination['lat'], destination['lng'])

    #Using the "places_nearby" API to get tourist attractions, within a 5000m radius, of the latitude-longitude of the chosen destination
    places_result = googlemapsapi.places_nearby(location=lat_lng,radius=5000,type='tourist_attraction')
    return [place['name'] for place in places_result['results']]

In [7]:
def get_weather(destination):
    """Fetches the current weather description for a given destination using OpenWeatherMap API.

    Args:
        destination (str): The city name for which to retrieve the weather.

    Returns:
        str: A string describing the current weather conditions (e.g., 'clear sky').
             Note: This function assumes the API key is stored in secrets
             and does not handle potential errors like invalid destinations,
             network issues, or API key problems gracefully. It might raise
             exceptions (e.g., KeyError, requests.exceptions.RequestException)
             if the API call fails or the response format is unexpected. Requires
             'requests' libraries to be imported.
    """
    
    api_key = openweather_api_key
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={destination['lat']}&lon={destination['lng']}&appid={api_key}"
    response = requests.get(url).json()
    if response.get('weather'):
        return response['weather'][0]['description']
    else:
        return "Weather data not available."

In [8]:
def generate_itinerary(destination, start_date, end_date, budget, interests):
    """Generates a personalized travel itinerary using OpenAI.

    Fetches local attractions and weather forecasts to create a detailed,
    day-by-day plan based on user preferences and constraints.

    Args:
        destination (str): The city or region for the trip.
        start_date (str): The starting date of the trip (e.g., "DD-MM-YYYY").
        end_date (str): The ending date of the trip (e.g., "DD-MM-YYYY").
        budget (float | int): The total budget for the trip in GBP (£).
        interests (list[str]): A list of the traveler's interests (e.g., ["history", "food"]).

    Returns:
        str: A string containing the generated day-by-day itinerary from the OpenAI model.

    Raises:
        openai.APIError: If there is an issue communicating with the OpenAI API.
        # Note: This function also depends on the successful execution of
        # get_attractions() and get_weather(), which might raise their own specific errors
        # (e.g., related to Google Maps API, OpenWeatherMap API, network issues, or missing keys).

    Note:
        Assumes `get_attractions` and `get_weather` functions are defined and
        accessible in the same scope and handle their own API interactions.
        Also for successful execution of geoencoding, this function requires 
        the 'googlemaps' libraries to be installed and a valid Google Maps API key 
        stored in secrets under the key "googlemaps_api_key".
    """

    #Generating the latitude and longitudes for the chosen destination using GoogleMaps APIs    
    gmaps = googlemaps.Client(key=googlemaps_api_key)
    geocode_result = gmaps.geocode(destination)
    if not geocode_result:
        raise ValueError(f"Could not geocode destination: {destination}")
    location = geocode_result[0]['geometry']['location']
    
    tourist_attraction = get_attractions(gmaps, location) #Getting tourist attractions for the chosen destination
    weather_forecast = get_weather(location) #Getting the weather forecast, for the chosen destination, and chosen dates.

    client = OpenAI(api_key=OPENAIKEY)
    prompt = f"""
    You are a helpful tour planner.
    Please create a day-by-day itinerary, within 1000 words or less,
    for a trip to {destination},
    from {start_date} to {end_date},
    within a budget of £{budget}, and 
    with focus on the below interests {', '.join(interests)}.
    Please also include places like, {tourist_attraction}, in the itinerary and
    factor the forecasted weather, like {weather_forecast} while building the itinerary.
    """
    response = client.chat.completions.create(model="gpt-3.5-turbo-0125",
                                              messages=[
                                                            {
                                                                "role": "developer",
                                                                "content": prompt
                                                            }
                                                        ],
                                              max_tokens=1000,
                                              temperature=0)
    return response.choices[0].message.content

In [9]:
destination = input("Enter your destination:")

Enter your destination: Paris


In [10]:
budget = input("Enter your budget (£):")

Enter your budget (£): 500


In [11]:
# Create the Start-Date DatePicker widget
startdatewidget = widgets.DatePicker(description='Pick a Start Date',
                                     style={'description_width': 'initial'})

# Create the End-Date DatePicker widget
enddatewidget = widgets.DatePicker(description='Pick an End Date',
                                   style={'description_width': 'initial'})

# Define your interests
options = ["Nature", "History", "Food", "Adventure", "Shopping", "Relaxation"]

# Create the Interests SelectMultiple widget
interestswidget = widgets.SelectMultiple(options=options,
                                         description="Choose your interests:",
                                         rows=6,
                                         style={'description_width': 'initial'},
                                         disabled=False)

# Create a button
submit_button = widgets.Button(description="Submit")

# Output area to display selected items
output = widgets.Output()

In [12]:
# On button click
def on_submit_clicked(b):
    global itinerary
    with output:
        clear_output()
        start_date = startdatewidget.value
        end_date = enddatewidget.value
        interests = list(interestswidget.value)

        if not start_date or not end_date or not interests:
            print("❗ Please select both trip start and end dates, and at least one interest, before submitting.")
        else:
            print("Generating your trip plan...")
            itinerary = generate_itinerary(destination, start_date, end_date, budget, interests)
            print("Your AI-generated itinerary:\n ")
            print(itinerary)

submit_button.on_click(on_submit_clicked)

# Display everything
display(startdatewidget, enddatewidget, interestswidget, submit_button, output)

DatePicker(value=None, description='Pick a Start Date', step=1, style=DescriptionStyle(description_width='init…

DatePicker(value=None, description='Pick an End Date', step=1, style=DescriptionStyle(description_width='initi…

SelectMultiple(description='Choose your interests:', options=('Nature', 'History', 'Food', 'Adventure', 'Shopp…

Button(description='Submit', style=ButtonStyle())

Output()

## Now let us add some conversational capabilities to this AI trip-planner

In [14]:
history = ChatMessageHistory() #Using LanChain's ChatMessageHistory to save Chat session history
#Using OpenAI "gpt-3.5-turbo-0125" model to generate the itinenary
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=0, 
                 api_key=OPENAIKEY)

# Passing the generated itinenary to the Chat history
history.add_ai_message(itinerary)

In [15]:
# Generating a new prompt template to handle conversation history
prompt = ChatPromptTemplate.from_messages([("system", "You are a helpful travel planning assistant. Refine the itinerary as per the user's requests."),
                                           ("ai", "{history}"),
                                           ("human", "{input}"),])


def get_refined_reply(user_input):
    # Add the user's new request to memory
    history.add_user_message(user_input)

     # Format the conversation so far
    conversation = ""
    for msg in history.messages:
        if msg.type == "human":
            conversation += f"User: {msg.content}\n"
        else:
            conversation += f"AI: {msg.content}\n"

    # Create the chain and run it
    chain = LLMChain(llm=llm, prompt=prompt)
    refined_itinerary = chain.run(history=conversation, input=user_input)
    history.add_ai_message(refined_itinerary)
    return refined_itinerary

In [16]:
# The below code will continue to ask the end-user if he/she wishes to add refinements to the generated itinenary till he/she enters 'Exit'
user_refined_query = "Continue"
query1 = "Do you want some changes (e.g. 'Add more food experiences on Day 2') ? If No, enter 'Exit' to quit"
query2 = "Do you want some more changes? If No, enter 'Exit' to quit"

while user_refined_query != 'Exit':
    user_refined_query = input(query1)        
    if not user_refined_query == "Exit": # if user wants some refinement
        print("Generating your refined trip plan...")
        refined_itinerary = get_refined_reply(user_refined_query)
        print("Your refined AI-generated itinerary:\n ")
        print(refined_itinerary)
        query1 = query2
    else:
        # check if user has entered 'Exit' or not
        # In case of some invalid input, remind user to input some request for itinenary refinement or enter 'Exit' to quit.
        if not user_refined_query == "Exit":
            print("❗ Please provide some input, if you wish to refine the itinenary, or enter 'Exit' to quit")

Do you want some changes (e.g. 'Add more food experiences on Day 2') ? If No, enter 'Exit' to quit Add more food experiences on Day 3


Generating your refined trip plan...


/tmp/ipykernel_31/2587822275.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_31/2587822275.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  refined_itinerary = chain.run(history=conversation, input=user_input)


Your refined AI-generated itinerary:
 
AI: Day 3: 2025-04-22 (Clear sky)
- Morning: Visit the Louvre Museum and see iconic artworks like the Mona Lisa.
- Afternoon: Stroll through the Jardin du Luxembourg and enjoy a picnic.
- Additional Food Experience: Indulge in some delicious French pastries at a local bakery or patisserie.
- Evening: Shopping at the upscale boutiques on Rue Saint-Honoré.

Would you like to add any specific type of food experience or cuisine for Day 3?


Do you want some more changes? If No, enter 'Exit' to quit Exit
